## Writing Partitions Concurrently

In [ ]:
# If True will increase the parallelism level during the partitions writing. It will decrease the
# writing time and increase memory usage.

In [ ]:
%reload_ext memory_profiler

import awswrangler as wr


In [ ]:
# Enter your bucket name

In [ ]:
import getpass
bucket = getpass.getpass()
path = f"s3://{bucket}/data/"

In [ ]:
# Reading 4 GB of CSV 

In [ ]:
noaa_path = "s3://noaa-ghcn-pds/csv/by_year/193"

cols = ["id", "dt", "element", "value", "m_flag", "q_flag", "s_flag", "obs_time"]
dates = ["dt", "obs_time"]
dtype = {x: "category" for x in ["element", "m_flag", "q_flag", "s_flag"]}

df = wr.s3.read_csv(noaa_path, names=cols, parse_dates=dates, dtype=dtype)

df["year"] = df["dt"].dt.year

print(f"Number of rows: {len(df.index)}")
print(f"Number of columns: {len(df.columns)}")

In [ ]:
# Default Writing

In [ ]:
%%time
%%memit

wr.s3.to_parquet(
    df=df,
    path=path,
    dataset=True,
    mode="overwrite",
    partition_cols=["year"],
)

In [ ]:
# Concurrent Partitioning (Decreasing writing time, but increasing memory usage)

In [ ]:
%%time
%%memit

wr.s3.to_parquet(
    df=df,
    path=path,
    dataset=True,
    mode="overwrite",
    partition_cols=["year"],
    concurrent_partitioning=True  # <-----
)